<a href="https://colab.research.google.com/github/ai-portfolio/deep_reinforcement_learning_stock_trading_bot/blob/master/deep_reinforcement_learning_stock_trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning Stock Trading Network
Stock trading bot built using deep Q-networks

author: Neal Gilmore   


---
### NOTE: Please do NOT trade using this application. This is for demonstration purposes ONLY! Any trading is done at your own risk and I will not be held liable for any losses.
---

## STEP 1: Install dependencies and setup environment

In [0]:
# install required libraries
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 51kB/s 
     |████████████████████████████████| 419kB 49.2MB/s 
     |████████████████████████████████| 3.0MB 48.2MB/s 


In [0]:
!pip install pandas-datareader

## STEP 2: Import required libraries and dependencies

In [48]:
# import required libraries
try:
  
  # standard libraries
  import math
  import random
  import numpy as np
  import pandas as pd
  import tensorflow as tf
  import matplotlib.pyplot as plt
  
  # stock market reader
  import pandas_datareader as data_reader
  
  from tqdm import tqdm_notebook, tqdm
  from collections import deque
  
except Exception as e:
  print('>> An error occurred loading one or more libraries\n')
  print(e)
  
else:
  print('>> Required libraries installed successfully')

>> Required libraries installed successfully


In [49]:
# check the version of tensorflow
print(tf.__version__)

1.14.0


## STEP 3: Build the AI Trading Network

In [0]:
class AI_Trader():
  '''
    Class used to represent the AI_Trader.
  '''
  
  def __init__(self,
               state_size,
               action_space=3,
               model_name='AI_Trader'): # hold, buy, sell
    '''
    This function initializes the AI_Trader class.
    
    @param state_size: int,
    @param action_space: int,
    @param model_name: str, name of the model
    '''
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    '''
    This function defines the neural network model architecture.
    '''
    
    # hidden layers
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32, 
                                    activation='relu', 
                                    input_dim=self.state_size))
    model.add(tf.keras.layers.Dense(units=64, 
                                    activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, 
                                    activation='relu'))
    
    # output layer
    model.add(tf.keras.layers.Dense(units=self.state_size, 
                                    activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
    
  def trade(self, state):
    '''
    This function defines the trade based on the state.
    
    It takes a state as an input and generates a random number. If that random 
    number is <= espilon, then return a randomly generated number between 0-2 
    (based on the action_state). Else, call the model and perform a prediction 
    based on the input state, returning the action with the highest probability 
    (likelihood) between the three options.
 
    
    @param state: should use random generated action or the model to perform 
                  the action
    '''
    
    if random.random() <= self.epsilon: 
      return random.randrange(self.action_space) # the stop point 
    
    actions = self.model.predict(state)
    
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    '''
    This function defines a custom training function.
    
    It trains the model based on a specified batch of data
    
    @param batch_size: int, size of each batch of data
    '''
    
    batch = []
    
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
      for state, action, reward, next_state, done in batch: 
        reward = reward 
        
        if not done: 
          reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
          
        target = self.model.predict(state)
        target[0][action] = reward
        
        self.model.fit(state, 
                       target, 
                       epochs=1, 
                       verbose=0)
        
      if self.epsilon > self.epsilon_final: 
        self.epsilon *= self.epsilon_decay

## STEP 4: Data preprocessing

### Define helper functions

#### Sigmoid function

In [0]:
def sigmoid(x):
  '''
  This function is the activation function used at the end of the neural network
  for binary classification - scaled to 0-1 to remove the effect of stock price.
  
  @param x: float32, stock price
  '''
  
  return 1 / (1 + math.exp(-x))

#### Price format function

In [0]:
def stock_price_format(n):
  '''
  This function formats the stock price. If negative, add a '-' in front of it
  and, in either case, limits the output to 2 decimal places
  
  @param n: float32, stock price
  '''
  
  if n < 0:
    return '- $ {0:2f}'.format(abs(n))
  else:
    return '$ {0:2f}'.format(abs(n))

#### Data loader

In [0]:
def data_loader(stock_name):
  '''
  This function
  
  @param stock_name: str, name of the stock to load into memory
  '''
  
  dataset = data_reader.DataReader(stock_name, data_source='yahoo')
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']
  
  return close

In [0]:
dataset = data_reader.DataReader('AAPL', data_source='yahoo')

In [55]:
dataset.head(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,26.782711
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,26.829010
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,26.402260
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,26.353460
2010-01-08,30.285715,29.865715,30.042856,30.282858,111902700.0,26.528664
2010-01-11,30.428572,29.778572,30.400000,30.015715,115557400.0,26.294640
2010-01-12,29.967142,29.488571,29.884285,29.674286,148614900.0,25.995537
2010-01-13,30.132856,29.157143,29.695715,30.092857,151473000.0,26.362217
2010-01-14,30.065714,29.860001,30.015715,29.918571,108223500.0,26.209538


#### State creator function

In [0]:
def state_creator(data, timestep, window_size):
  '''
  This function takes the stock data and generates states from it
  
  @param data: dataframe, stock market data previously downloaded with data_loader
  @param timestamp: datetime, timestamp of the stock to predict for
  @param window_size: int, number of previous days' stock prices to include in
                           the prediction
  '''
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

#### Load the dataset

In [0]:
stock_name = 'AAPL'

data = data_loader(stock_name)

## STEP 5: Train the AI Trader

### Set hyperparameters

In [0]:
window_size = 10 # number of previous days' stock prices to use
episodes = 1000 # number of times to run the algorithm
batch_size = 32
data_samples = len(data) - 1 # have to remove the last day's stock price as it is what is being predicted

### Define the Trader model

In [0]:
trader = AI_Trader(window_size)

In [66]:
# display the architecture of the model
trader.model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_16 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_17 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_18 (Dense)             (None, 10)                1290      
Total params: 12,074
Trainable params: 12,074
Non-trainable params: 0
_________________________________________________________________


### Training loop

In [0]:
for episode in range(1, episodes + 1):
  
  print('Episode: {}/{}'.format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = [] # ensure inventory is clean at the start each time
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0 
    
    if action == 1: # buying stock
      trader.inventory.append(data[t])
      print('AI Trader bought: {}'.format(stock_price_format(data[t])))
    
    elif action == 2 and len(trader.inventory) > 0: # selling stock
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print('AI Trader sold: {} Profit: {}'.format(stock_price_format(data[t]), stock_price_format(data[t] - buy_price)))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print('####################')
      print('TOTAL PROFIT: {}'.format(total_profit))
      print('####################')
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episodes % 10 == 0:
    trader.model.save('ai_trader_{}.h5'.format(episode))

  0%|          | 0/2388 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought: $ 30.625713
AI Trader sold: $ 30.082857 Profit: - $ 0.542856
AI Trader bought: $ 30.015715
AI Trader sold: $ 30.092857 Profit: $ 0.077143
AI Trader bought: $ 28.250000
AI Trader sold: $ 29.010000 Profit: $ 0.760000
AI Trader bought: $ 29.420000
AI Trader bought: $ 29.697144
AI Trader bought: $ 28.469999
AI Trader bought: $ 27.437143
AI Trader bought: $ 27.980000
AI Trader sold: $ 28.461428 Profit: - $ 0.958572
AI Trader bought: $ 27.922857
AI Trader sold: $ 28.027143 Profit: - $ 1.670000
AI Trader sold: $ 27.874287 Profit: - $ 0.595713
AI Trader bought: $ 28.381428
AI Trader sold: $ 28.625713 Profit: $ 1.188570
AI Trader bought: $ 28.990000
AI Trader bought: $ 28.809999


  1%|▏         | 33/2388 [00:08<09:45,  4.02it/s]

AI Trader bought: $ 28.631428


  1%|▏         | 34/2388 [00:11<47:52,  1.22s/it]

...

 48%|████▊     | 1152/2388 [1:03:56<1:09:58,  3.40s/it]

AI Trader bought: $ 96.129997


 48%|████▊     | 1153/2388 [1:04:00<1:10:04,  3.40s/it]

AI Trader sold: $ 95.589996 Profit: - $ 0.540001


 49%|████▊     | 1160/2388 [1:04:24<1:09:39,  3.40s/it]